# Munging the inaugural addresses


In [88]:
import urllib.request
# html = urllib.request.urlopen('http://avalon.law.yale.edu/subject_menus/inaug.asp', data = None)

In [89]:
html = urllib.request.urlopen('http://avalon.law.yale.edu/subject_menus/inaug.asp', data = None)

In [90]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())
print(html)

<html>
 <head>
  <link href="../css/site.css" rel="stylesheet" type="text/css">
   <title>
    Avalon Project - The Inaugural Addresses of the Presidents
   </title>
  </link>
 </head>
 <body>
  <div class="HeaderContainer">
   <ul class="HeaderTopTools">
    <li class="Search">
     <table border="0" cellpadding="0" cellspacing="0">
      <form action="http://www.google.com/u/yaleu" id="form" method="get" name="form" target="_blank">
       <input name="domains" type="hidden" value="yale.edu">
        <input name="sitesearch" type="hidden" value="avalon.law.yale.edu/">
         <tr>
          <td>
           <!--<input onblur="if (this.value==''){this.value='Search Avalon'}" onfocus="if (this.value=='Search Avalon'){this.value=''}" type="text" name="q" value="Search Avalon" alt="Search Avalon" class="SiteSearchBox" />
				-->
          </td>
          <!--<td><a href="javascript:this.form.submit();"><img src="http://www.law.yale.edu/library/searchGo.gif" alt="Begin Search" /></a></td>

In [91]:
soup.title

<title>Avalon Project - The Inaugural Addresses of the Presidents</title>

In [92]:
links = []
dates = []
pres = []
url_prefix = 'http://avalon.law.yale.edu/'
import re
for link in soup.find_all('a'):
    raw_link = link.get('href')
#     print(raw_link)
    if re.search('subject_menus', raw_link):
        continue
    elif re.search('about/purpose', raw_link):
        continue
    elif re.search('default', raw_link):
        continue
    elif re.match('http', raw_link):
        continue            
    else:
        links.append(url_prefix + raw_link[3:])
        dates.append(link.text)
        pres.append(link.find_previous('span').get_text())
    
print(links)
print(dates)
print(pres)

['http://avalon.law.yale.edu/18th_century/wash1.asp', 'http://avalon.law.yale.edu/18th_century/wash2.asp', 'http://avalon.law.yale.edu/18th_century/adams.asp', 'http://avalon.law.yale.edu/19th_century/jefinau1.asp', 'http://avalon.law.yale.edu/19th_century/jefinau2.asp', 'http://avalon.law.yale.edu/19th_century/madison1.asp', 'http://avalon.law.yale.edu/19th_century/madison2.asp', 'http://avalon.law.yale.edu/19th_century/monroe1.asp', 'http://avalon.law.yale.edu/19th_century/monroe2.asp', 'http://avalon.law.yale.edu/19th_century/qadams.asp', 'http://avalon.law.yale.edu/19th_century/jackson1.asp', 'http://avalon.law.yale.edu/19th_century/jackson2.asp', 'http://avalon.law.yale.edu/19th_century/vanburen.asp', 'http://avalon.law.yale.edu/19th_century/harrison.asp', 'http://avalon.law.yale.edu/19th_century/polk.asp', 'http://avalon.law.yale.edu/19th_century/taylor.asp', 'http://avalon.law.yale.edu/19th_century/pierce.asp', 'http://avalon.law.yale.edu/19th_century/buchanan.asp', 'http://aval

In [93]:
import pandas as pd

In [94]:
urls = pd.Series(links)
years = pd.Series(dates)
pres = pd.Series(pres)

inaugurals = pd.concat([urls, years, pres], axis = 1)
inaugurals.columns = ['urls', 'years', 'pres']
inaugurals.head()

,urls,years,pres
0,http://avalon.law.yale.edu/18th_century/wash1.asp,1789,George Washington
1,http://avalon.law.yale.edu/18th_century/wash2.asp,1793,George Washington
2,http://avalon.law.yale.edu/18th_century/adams.asp,1797,John Adams
3,http://avalon.law.yale.edu/19th_century/jefina...,1801,Thomas Jefferson
4,http://avalon.law.yale.edu/19th_century/jefina...,1805,Thomas Jefferson


In [95]:
pres_list = []
for span in soup.find_all('span'):
#     pres = span.get_text()
    pres = span.contents
#     pres_list.append(pres)
    pres_list = pres_list + pres
print(pres_list)

['George Washington', 'John Adams', 'Thomas Jefferson', 'James Madison', 'James Monroe', 'John Quincy Adams', 'Andrew Jackson', 'Martin Van Buren', 'William Henry Harrison', 'James Polk', 'Zachary Taylor', 'Franklin Pierce', 'James Buchanan', 'Abraham Lincoln', 'Ulysses S. Grant', 'Rutherford B. Hayes', 'James A. Garfield', 'Grover Cleveland', 'Benjamin Harrison', 'William McKinley', 'Theodore Roosevelt', 'William Howard Taft', 'Woodrow Wilson', 'Warren G. Harding', 'Calvin Coolidge', 'Herbert Hoover', 'Franklin D. Roosevelt', 'Harry S. Truman', 'Dwight D. Eisenhower', 'John F. Kennedy', 'Lyndon Baines Johnson', 'Richard Milhous Nixon', 'Jimmy Carter', 'Ronald Reagan', 'George Bush', 'William Clinton', 'George W. Bush', 'Barack Obama']


## Now get the texts
Here's where we go in and scrape the texts and add them to a column in our dataframe.

In [96]:
# A function to count the words
def wordcounter(text):
    if "This one was funny" in text:
        count = "N/A"
    else:
        words = text.split()
        count = 0
        for word in words:
            count += 1
    return count
        

In [97]:
# A function to strip the texts
def format_text(url):
    html = urllib.request.urlopen(url, data = None)
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text()
    text = text.strip('/n')
    print(text[:50])
    return text

In [100]:
def remove_head(text):
    text_words_list = text.split()
    start_pos = text_words_list.index("2000")
    print(start_pos)
    sequence = ["Inaugural", "Speeches", "Page"]
    for i in range(len(text_words_list)):
        if text_words_list[i:i+len(sequence)] == sequence:
            end_pos = i
    print(end_pos)
    text_words_list = text_words_list[start_pos:end_pos]
    text.join(text_words_list) # this is wrong
    return text
    print(text)

def remove_str_head(text):
    try:
        start_pos = text.index("21st Century 2000 - ") + len("21st Century 2000 - ")
        end_pos = text.index("Inaugural Speeches Page")
        return(text[start_pos:end_pos])
    except:
        return text


In [101]:
import time
texts = []
fail_count = 0
for url in inaugurals['urls']:
    print(url)
    try:
        scrape = format_text(url)
        scrape = remove_str_head(scrape)
        texts.append(scrape)
#         html = urllib.request.urlopen(url, data = None)
#         soup = BeautifulSoup(html, "html.parser")
#         text = soup.get_text()
#         text = text.strip()
#         texts.append(text)
    except:
        texts.append("This one was funny:" + url)
        fail_count += 1
    time.sleep(3)

print(fail_count)


http://avalon.law.yale.edu/18th_century/wash1.asp


The Avalon Project : First Inaugural Address of 
http://avalon.law.yale.edu/18th_century/wash2.asp


The Avalon Project : Second Inaugural Address of
http://avalon.law.yale.edu/18th_century/adams.asp



Avalon Home
Document Collections
Ancient 4000bc
http://avalon.law.yale.edu/19th_century/jefinau1.asp


The Avalon Project : Jefferson's First Inaugural
http://avalon.law.yale.edu/19th_century/jefinau2.asp


The Avalon Project : Jefferson's Second Inaugura
http://avalon.law.yale.edu/19th_century/madison1.asp


The Avalon Project : Madison's First Inaugural A
http://avalon.law.yale.edu/19th_century/madison2.asp


The Avalon Project : Madison's Second Inaugural 
http://avalon.law.yale.edu/19th_century/monroe1.asp


The Avalon Project : First Inaugural Address of 
http://avalon.law.yale.edu/19th_century/monroe2.asp


The Avalon Project : Second Inaugural Address of
http://avalon.law.yale.edu/19th_century/qadams.asp


The Avalon Project : I

In [102]:
# Remove the superfluous text before and after the addresses 
## I don't use this function because I built it into the scrape process above.

# clean_texts = []
# for text in texts:
#     try:
#         text = remove_str_head(text)
#         clean_texts.append(text)
#     except:
#         clean_texts.append("N/A")


In [103]:
# split and count the words in the texts


In [104]:
counts = []
for text in texts:
    counts.append(wordcounter(text))
    
print(counts)

[1443, 149, 2417, 1742, 2162, 1193, 1230, 3503, 4600, 2919, 1134, 1181, 'N/A', 8434, 4805, 1096, 3335, 'N/A', 3770, 841, 1132, 1346, 2483, 'N/A', 1689, 2024, 4401, 3967, 2226, 992, 5438, 1709, 1536, 3336, 'N/A', 3758, 1887, 1811, 1329, 557, 2276, 2452, 1647, 1350, 1490, 2115, 1797, 1232, 2438, 2564, 2319, 1601, 2176, 1595, 2073, 2417]


In [105]:
# now add the new list to the dataframe

inaugurals['texts'] = texts
inaugurals['wordcounts'] = counts

In [106]:
inaugurals.head()

,urls,years,pres,texts,wordcounts
0,http://avalon.law.yale.edu/18th_century/wash1.asp,1789,George Washington,\n\n\n\n\r\n\r\nFirst Inaugural Address of Geo...,1443
1,http://avalon.law.yale.edu/18th_century/wash2.asp,1793,George Washington,\n\n\n\n\r\n\r\n\r\nSecond Inaugural Address o...,149
2,http://avalon.law.yale.edu/18th_century/adams.asp,1797,John Adams,\n\n\n\n\n© 2008 Lillian Goldman Law Library\n...,2417
3,http://avalon.law.yale.edu/19th_century/jefina...,1801,Thomas Jefferson,\n\n\n\n\nThe Avalon Project at Yale Law Scho...,1742
4,http://avalon.law.yale.edu/19th_century/jefina...,1805,Thomas Jefferson,\n\n\n\n\r\n\r\nThomas Jefferson Second Inaugu...,2162


In [108]:
for text in inaugurals['texts']:
    print(text[:50])
    print(type(text))







First Inaugural Address of George Washingt
<class 'str'>







Second Inaugural Address of George Washi
<class 'str'>





© 2008 Lillian Goldman Law Library
127 Wall S
<class 'str'>





The Avalon Project  at Yale Law School
Thoma
<class 'str'>






Thomas Jefferson Second Inaugural Address 
<class 'str'>





The Avalon Project  at Yale Law School
James 
<class 'str'>





The Avalon Project  at Yale Law School
James 
<class 'str'>


The Avalon Project : First Inaugural Address of 
<class 'str'>


The Avalon Project : Second Inaugural Address of
<class 'str'>







Inaugural Address of John Quincy Adams


<class 'str'>







First Inaugural Address of Andrew Jackso
<class 'str'>







Second Inaugural Address of Andrew Jacks
<class 'str'>
This one was funny:http://avalon.law.yale.edu/19th
<class 'str'>







Inaugural Address of William Henry Harri
<class 'str'>







Inaugural Address of James Knox Polk

TU
<class 'str'>







Inaugural Address of Zachary Taylor

MON


In [109]:
# Save the dataframe to csv
inaugurals.to_csv('data/inaugurals.csv')

In [20]:
# A function to strip the texts
def format_text(url):
    html = urllib.request.urlopen(url, data = None)
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text()
    text = text.strip('/n')
    print(text[:50])
    return text

In [21]:
# A function to count the words
def wordcounter(text):
    if "This one was funny" in text:
        count = "N/A"
    else:
        words = text.split()
        count = 0
        for word in words:
            count += 1
    return count
        